# Atomic C++ Data Types

Recall that C++ requires users to specify the **data type** of each variable. The **primary built-in** data types in C++ are: `int`, `float`, `double`, `bool`, and `char`. There is also a special type called `pointer` which holds a memory location. C++ also has a collection of [compound data types](https://www.learncpp.com/cpp-tutorial/introduction-to-compound-data-types/).

### Numeric data

These are `int`, `float`, and `double`. Note that there is no distinction between integer and float division in C++ (unlike in Python). That is, when two integers are divided, then integer division is done. Otherwise, it's float division whenever one of the numbers is a float. Exponentiation is done using `pow()` from the `cmath` library and the remainder (modulo) operator is done with `%`.

In [1]:
%%writefile ./code/02_01.cpp
#include <iostream>
#include <cmath>
using namespace std;

int main(){
    // order of operations
    cout << (2 + 3 * 4) << endl;
    cout << (2 + 3) * 4 << endl;

    // divisions
    cout << endl;
    cout << 7 / 3 << endl;
    cout << float(7) / 3 << endl;
    cout << 7. / 3 << endl;
    cout << 6. / 3 << endl;

    // modulo & pow
    cout << endl;
    cout << 7 % 3 << endl;
    cout << pow(2, 10) << endl;
    cout << pow(2, 100) << endl;

    return 0;
}

Overwriting ./code/02_01.cpp


In [2]:
from utils import run

run()

$ g++-15 ./code/02_01.cpp -o ./code/02_01
$ ./code/02_01

14
20

2
2.33333
2.33333
2

1
1024
1.26765e+30


**Remark.** Modifiers like `short`, `long`, and `unsigned` can optionally be used for numeric variables to help ensure that space is used as efficiently as possible.


## Boolean data

C++ uses the keyword `bool` which is not capitalized. The possible state values for a C++ Boolean are `true` and `false` (note: lower case). C++ uses the standard Boolean operators: `&&`, `||`, `!`. Note that the internally stored values representing `true` and `false` are actually `1` and `0` respectively. Indeed:

In [3]:
%%writefile ./code/02_02.cpp
#include <iostream>
using namespace std;

// function that demonstrates logical operators
int main() {
  cout << true  << endl;
  cout << false << endl;
  cout << (true || false) << endl;
  cout << (true && false) << endl;
  return 0;
}

Overwriting ./code/02_02.cpp


In [4]:
run()

$ g++-15 ./code/02_02.cpp -o ./code/02_02
$ ./code/02_02

1
0
1
0


Boolean data are also used as results for things like comparison (`<`, `==`, etc.). As usual, relational and logical operators can be combined to form complex logical conditions. The following function demonstrates relational operators:

In [15]:
%%writefile ./code/02_03.cpp
#include <iostream>
using namespace std;

int main(){
    cout << (5 == 10) << endl;
    cout << (10 > 5) << endl;
    cout << ((5 >= 1) && (5 <= 10)) << endl;
    cout << ((5 >= 1) || (1 / 0)) << endl;
    return 0;
}

Overwriting ./code/02_03.cpp


In [16]:
run()

$ g++-15 ./code/02_03.cpp -o ./code/02_03
$ ./code/02_03

0
1
1
1


**Remark.** Observe short circuit with OR also works since `1/0` was not evaluated.

In [17]:
%%writefile ./code/02_04.cpp
#include <iostream>
using namespace std;

//  Demonstrates how to use variables in C++
//  moreover, we assign an integer to a boolean variable,
//  showing the static typing of C++.
int main(){

    int theSum = 4;
    cout << theSum << endl;

    theSum = theSum + 1;
    cout << theSum << endl;

    bool theBool = true;
    cout << theBool << endl;

    theBool = 4;
    cout << theBool << endl;

    return 0;
}

Writing ./code/02_04.cpp


In [18]:
run()

$ g++-15 ./code/02_04.cpp -o ./code/02_04
$ ./code/02_04

4
5
1
1
